## Projet Societe Generale

*Léon Thomir - Irénée De Leusse - Amaury - Louis Bolzinger*  
CentraleSupélec 3A MMF - Société Générale

In [2]:
import numpy as np
from scipy import optimize
import pandas as pd
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from scipy.special import gamma
from math import pi
#pip install fbm==0.1.0
from fbm import FBM
import warnings
warnings.filterwarnings('ignore')

Import et cleaning de la data

In [3]:
file = "C:/Users/louis/OneDrive - De Vinci/CENTRALE SUPELEC/Projet/Data issuers.xlsx"
market_cap = pd.read_excel(file, sheet_name="Mod Market Cap")
market_cap = market_cap.set_index("Dates").loc['2019-10-28':'2020-10-13']
debt = pd.read_excel(file, sheet_name="Gross Debt", nrows=1)

## Probabilité de défault & Merton

### Méthodes

#### Modèle de Merton 'Classique'

In [5]:
def BSM(ticker, market_cap, debt, T=1, frequency=252, rf=0, epsilon=10e-5):
    company_debt = debt[[ticker]].iloc[0, 0]
    company_market_cap = market_cap[[ticker]].iloc[:, 0]
    current_time = 0
    equity_value = 0
    sigma_A = 0
    sigma_A_former = 0
    asset_values = []

    def d1_m(x, sigma_A, current_time):
        return ((np.log(x / company_debt)) + (rf + 0.5 * sigma_A ** 2) * current_time) / (
                sigma_A * np.sqrt(current_time))

    def d2_m(x, sigma_A, current_time):
        return d1_m(x, sigma_A, current_time) - sigma_A * np.sqrt(current_time)

    # inverse the black scholes formula
    def merton_formula(x, rf, current_time):
        d1_term = x * norm.cdf(d1_m(x, sigma_A, current_time))
        d2_term = company_debt * np.exp(-rf * current_time) * norm.cdf(d2_m(x, sigma_A, current_time))
        return d1_term - d2_term - equity_value

    sigma_E = np.std(np.diff(np.log(company_market_cap), n=1)) * np.sqrt(frequency)
    sigma_A = sigma_E

    while np.abs(sigma_A - sigma_A_former) > epsilon:

        asset_values = []

        for dt in range(company_market_cap.shape[0]):
            current_time = T + (frequency - dt - 1) / frequency
            equity_value = company_market_cap[dt]
            # find zero of Merton function, ie asset_value at the current_time
            asset_values.append(optimize.newton(merton_formula, company_debt, args=(rf, current_time)))

        # update of sigma_A and sigma_A_former
        sigma_A_former = sigma_A
        sigma_A = np.std(np.diff(np.log(asset_values), n=1)) * np.sqrt(frequency)

    # compute distance to default and default probability
    distance_to_default = d2_m(asset_values[-1], sigma_A, current_time)
    default_probability = (1 - norm.cdf(distance_to_default)) * 100

    distance_to_default_real = -d2_m(asset_values[-1], sigma_A, current_time)
    default_probability_real = norm.cdf(distance_to_default) * 100 # notation approximation : non default probability
    #print(distance_to_default_real)
    #print(default_probability_real)

    return sigma_A, distance_to_default, default_probability

#### Modèle de Merton avec Mouvement Brownien fractionnaire

In [6]:
def update_values_regression_fixed_intercept(Var, delta_t, sigma_A, iteration, plot=False):
    var_tau = np.array(Var)

    # Transformation logarithmique
    log_delta_t = np.log(delta_t)
    log_var_tau = np.log(var_tau)

    fixed_intercept_log_sigma2 = np.log(var_tau[0]) # assuming delta = 1 otherwise H is here

    # Régression linéaire
    X = log_delta_t.reshape(-1, 1)
    y = log_var_tau - fixed_intercept_log_sigma2

    model = LinearRegression(fit_intercept=False)
    model.fit(X, y)
    #print('Regression score for H is ',model.score(X, y))

    # Coefficients de la régression
    slope = model.coef_[0]

    # Calcul de H
    H = slope / 2
    sigma_A_former = sigma_A
    sigma_A = np.sqrt(var_tau[0]) * ((int(252/delta_t[0]))**(H))

    if plot:
        plt.scatter(log_delta_t, y, label='Données')
        plt.plot(log_delta_t, model.predict(log_delta_t.reshape(-1, 1)), color='red', label='Régression linéaire')
        plt.xlabel('log(Delta t)')
        plt.ylabel('log(Var(tau(Delta t)))')
        plt.title(f"Régression de l'itération {iteration}")
        plt.legend()
        plt.show()

    return sigma_A, sigma_A_former, H

#d2_hurst is used to calculate the default probability
def d1_hurst(x, sigma_A, t, T ,H, rf, company_debt):
    return ((np.log(x / company_debt)) + rf * (T-t) + (0.5 * sigma_A ** 2) * (T ** (2 * H) - t ** (2 * H))) / (
            sigma_A * (T - t)**H)
def d2_hurst(x, sigma_A, t, T, H, rf, company_debt):
    return d1_hurst(x, sigma_A, t, T, H, rf, company_debt) - sigma_A * (T - t)**H

#### Adapted Merton Formula From Necula

In [19]:
#d1 and d2 from Necula
def d1(x, sigma_A, t, T, H, rf, company_debt):
    return ((np.log(x / company_debt)) + rf * (T - t) + 0.5 * sigma_A ** 2 * (T ** (2 * H) - t ** (2 * H)) / (
            sigma_A * np.sqrt(T ** (2 * H) - t ** (2 * H))))


def d2(x, sigma_A, t, T, H, rf, company_debt):
    return ((np.log(x / company_debt)) + rf * (T - t) - 0.5 * sigma_A ** 2 * (T ** (2 * H) - t ** (2 * H)) / (
            sigma_A * np.sqrt(T ** (2 * H) - t ** (2 * H))))


# inverse the black scholes formula with Necula's expresions for d1 and d2
def merton_formula(x, rf, t, T, H, company_debt, equity_value, sigma_A):
    d1_term = x * norm.cdf(d1(x, sigma_A, t, T, H, rf, company_debt))
    d2_term = company_debt * np.exp(-rf * (T - t)) * norm.cdf(d2(x, sigma_A, t, T, H, rf, company_debt))
    return d1_term - d2_term - equity_value


def BSM_H(ticker, market_cap, debt, T=1, delta=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], rf=0, epsilon=10e-3, H0=0.5):
    frequency = []
    for d in delta:
        frequency.append(252 // d)
    company_debt = debt[[ticker]].iloc[0, 0]
    company_market_cap = market_cap[[ticker]].iloc[:, 0]
    sigma_A_former = 0
    H = H0
    sigma_E = np.std(np.diff(np.log(company_market_cap), n=1)) * np.sqrt(frequency[0])
    sigma_A = sigma_E

    n_iter = 1
    while np.abs(sigma_A - sigma_A_former) > epsilon:
        #print("Iteration ", n_iter)
        asset_values = {}
        for f in frequency:
            fasset_values = []
            n = company_market_cap.shape[0]
            days = []
            for i in range(n):
                if i % (n // f) == 0:
                    days.append(i)
            for day in days:
                t = day / n
                equity_value = company_market_cap[day]
                # find zero of Merton function, ie asset_value at the current_time
                fasset_values.append(optimize.newton(merton_formula, company_debt,
                                                     args=(rf, t, 1 + T, H, company_debt, equity_value, sigma_A),
                                                     maxiter=100))
            asset_values[f] = fasset_values

        # update values
        Var = []
        for i, f in enumerate(frequency):
            Var.append(np.var(np.diff(np.log(asset_values[f]), n=1)) )# *f)

        Mean = []
        for i, f in enumerate(frequency):
            Mean.append(np.mean(np.diff(np.log(asset_values[f]), n=1)))# *f)

        n_iter += 1
        #print("update values")
        sigma_A, sigma_A_former, H = update_values_regression_fixed_intercept(Var, delta, sigma_A, n_iter, False)
        #print(f"sigma= {sigma_A}, H={H}")

    assert len(Mean) == len(delta)
    mu = [round((Mean[k] + ((sigma_A**2)/(2*t)) * ((t+delta[k])**(2*H+1) - t**(2*H+1) - delta[k]**(2*H+1)) / (2*H+1)) / delta[k], 3) for k in range(len(delta))]
    #print(mu)
    # compute distance to default and default probability
    t = 1
    distance_to_default = d2_hurst(asset_values[frequency[0]][-1], sigma_A, t, t + T, H, mu[-1], company_debt)
    default_probability = (1 - norm.cdf(distance_to_default)) * 100
    return distance_to_default, default_probability, H, sigma_A, sigma_A_former,


#### Adapted Merton formula from Rostek

In [18]:

def ro_h(H):
    if H!=0.5:
        return( (np.sin(pi*(H-0.5))/(pi*(H-0.5)))*((gamma(1.5-H)**2)/(gamma(2-2*H))) )
    return( (gamma(1.5-H)**2)/(gamma(2-2*H)))


def d1_rostek(x, sigma_A, t, T, H, rf, company_debt, roH):
    return(
       ((np.log(x / company_debt)) + rf * (T-t) + 0.5* roH * ((sigma_A )** 2 )* ((T-t)**(2*H)))
       /(np.sqrt(roH)*sigma_A*((T-t)**H))
    )

def d2_rostek(x, sigma_A, t, T, H, rf, company_debt, roH):
    return(
        d1_rostek(x, sigma_A, t, T, H, rf, company_debt, roH) - np.sqrt(roH)*sigma_A*((T-t)**H)
    )


def merton_formula_rostek(x, rf, t, T, H, company_debt, equity_value, sigma_A, roH):
    d1_term=x * norm.cdf(d1_rostek(x, sigma_A, t, T, H, rf, company_debt, roH))
    d2_term=company_debt * np.exp(-rf * (T - t)) * norm.cdf(d2_rostek(x, sigma_A, t, T, H, rf, company_debt, roH))
    return (d1_term - d2_term - equity_value)


def BSM_H_rostek(ticker, market_cap, debt, T=1, delta=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], rf=0, epsilon=10e-3, H0=0.5):
    frequency = []
    for d in delta:
        frequency.append(252 // d)
    company_debt = debt[[ticker]].iloc[0, 0]
    company_market_cap = market_cap[[ticker]].iloc[:, 0]
    sigma_A_former = 0
    H = H0
    roH= ro_h(H)
    sigma_E = np.std(np.diff(np.log(company_market_cap), n=1)) * np.sqrt(frequency[0])
    sigma_A = sigma_E

    n_iter = 1
    while np.abs(sigma_A - sigma_A_former) > epsilon:
        #print("Iteration ", n_iter)
        asset_values = {}
        for f in frequency:
            fasset_values = []
            n = company_market_cap.shape[0]
            days = []
            for i in range(n):
                if i % (n // f) == 0:
                    days.append(i)
            for day in days:
                t = day / n
                equity_value = company_market_cap[day]
                # find zero of Merton function, ie asset_value at the current_time
                fasset_values.append(optimize.newton(merton_formula_rostek, company_debt,
                                                     args=(rf, t, 1 + T, H, company_debt, equity_value, sigma_A,roH),
                                                     maxiter=100))
            asset_values[f] = fasset_values

        # update values
        Var = []
        for i, f in enumerate(frequency):
            Var.append(np.var(np.diff(np.log(asset_values[f]), n=1)) )# *f)

        Mean = []
        for i, f in enumerate(frequency):
            Mean.append(np.mean(np.diff(np.log(asset_values[f]), n=1)))# *f)

        n_iter += 1
        #print("update values")
        sigma_A, sigma_A_former, H = update_values_regression_fixed_intercept(Var, delta, sigma_A, n_iter, plot=False)
        roH= ro_h(H)
        #print(f"sigma= {sigma_A}, H={H}")

    assert len(Mean) == len(delta)
    mu = [round((Mean[k] + ((sigma_A**2)/(2*t)) * ((t+delta[k])**(2*H+1) - t**(2*H+1) - delta[k]**(2*H+1)) / (2*H+1)) / delta[k], 3) for k in range(len(delta))]
    #print(mu)
    # compute distance to default and default probability
    t = 1
    distance_to_default = d2_hurst(asset_values[frequency[0]][-1], sigma_A, t, t + T, H, mu[-1], company_debt)
    default_probability = (1 - norm.cdf(distance_to_default)) * 100
    return distance_to_default, default_probability, H, sigma_A, sigma_A_former


### **Implémentation**

In [24]:
def compute_proba_default(index_ticker = 0, maturity = [1, 2, 5, 10, 15], display_graphe = True, display_H_coeff = True, ticker = False, metric = 'default'): 
    if ticker == False :  
        ticker = market_cap.columns[index_ticker]

    proba_merton = np.zeros(len(maturity))
    proba_necula = np.zeros(len(maturity))
    proba_rostek = np.zeros(len(maturity))

    if metric == 'default': 
        index_merton = 2
        index_necula_rostek = 1
    elif metric == 'sigma': 
            index_merton = 0
            index_necula_rostek = 3 
    for i, m in enumerate(maturity):
        proba_merton[i] = BSM(ticker, market_cap, debt, T=m)[index_merton]
        proba_necula[i] = BSM_H(ticker, market_cap, debt, T=m)[index_necula_rostek]
        proba_rostek[i] = BSM_H_rostek(ticker, market_cap, debt, T=m)[index_necula_rostek]
    
    Hurst_coef = BSM_H(ticker, market_cap, debt, T=1)[2]

    if display_H_coeff :  print(f"{ticker} Hurst : {Hurst_coef}")
    if display_graphe : 
        plt.figure()
        plt.plot(maturity, proba_merton, label="Merton")
        plt.plot(maturity, proba_necula, label="Necula")
        plt.plot(maturity, proba_rostek, label="Rostek")
        plt.legend()
        plt.title(f"Proba de défaut selon la maturité T pour {ticker}")
        plt.xlabel("T")
        plt.ylabel("Proba")
        plt.show()
    
    return proba_merton, proba_necula, proba_rostek, Hurst_coef

def export_latex(liste_ticker, maturity = [1, 2, 5, 10], metric = 'default'): 
    for maturity in maturity: 
        export = ''
        if metric == 'default': 
            index_merton = 2
            index_necula_rostek = 1
        elif metric == 'sigma': 
            index_merton = 0
            index_necula_rostek = 3
        for ticker in liste_ticker: 
            export  += str(ticker)
            export += ' & '
            proba_merton = BSM(ticker, market_cap, debt, T=maturity)[index_merton]
            export += str(proba_merton) + ' & '
            proba_necula = BSM_H(ticker, market_cap, debt, T=maturity)[index_necula_rostek]
            export += str(proba_necula) + ' & '
            proba_rostek = BSM_H_rostek(ticker, market_cap, debt, T=maturity)[index_necula_rostek]
            export += str(proba_rostek) + ' & '
            Hurst_coef = BSM_H(ticker, market_cap, debt, T=1)[2]
            export += str(Hurst_coef) + ' \\ \n'
        print(export)

In [ ]:
compute_proba_default(ticker = 'FGR FP Equity', metric = 'default')

In [ ]:
liste_equity = ['FGR FP Equity', 
                'CRH LN Equity', 
                'FR FP Equity' , 
                'ADP FP Equity' , 
                'DAI GY Equity', 
                'VIE FP Equity', 
                'LHA GY Equity', 
                'CO FP Equity']
export_latex(liste_equity)

## Simulation

### Mouvement Brownien Fractionnaires

#### Méthodes

Comparaison avec des résultats simulés

In [2]:
def evol_va(mu,sigma_a,fbm_sample,times,H):
    VA=np.exp( mu*times - ((sigma_a**2)/2)*(times**(2*H)) + sigma_a*fbm_sample)
    return(VA)

def genhurst(S, q):
    """
    S : série temporelle
    q : ordre 
    
    """
    
    # Obtient la longueur de la série temporelle
    L = len(S)

    # Avertissement si la série temporelle est très courte (moins de 100 points)
    #if L < 100:
    #    warnings.warn('Data series very short!')

    # Initialisation d'un tableau H pour stocker les résultats du coefficient de Hurst
    H = np.zeros((len(range(5, 20)), 1))  # Crée un tableau de zéros de forme (15, 1)
    k = 0  # Initialise un compteur

    # Boucle sur différentes fenêtres de temps (Tmax)
    for Tmax in range(5, 20):

        # Génère une séquence de nombres de 1 à Tmax
        x = np.arange(1, Tmax + 1, 1)

        # Initialise un tableau mcord pour stocker les résultats locaux du coefficient de Hurst
        mcord = np.zeros((Tmax, 1))  # Crée un tableau de zéros de forme (Tmax, 1)

        # Boucle à travers les décalages temporels (tt) dans la fenêtre actuelle
        for tt in range(1, Tmax + 1):

            # Calcule les différences et les valeurs retardées
            dV = S[np.arange(tt, L, tt)] - S[np.arange(tt, L, tt) - tt]
            VV = S[np.arange(tt, L + tt, tt) - tt]
            N = len(dV) + 1
            X = np.arange(1, N + 1, dtype=np.float64)
            Y = VV

            # Calcul des coefficients pour ajuster une droite
            mx = np.sum(X) / N
            SSxx = np.sum(X**2) - N * mx**2
            my = np.sum(Y) / N
            SSxy = np.sum(np.multiply(X, Y)) - N * mx * my
            cc1 = SSxy / SSxx
            cc2 = my - cc1 * mx
            ddVd = dV - cc1
            VVVd = VV - np.multiply(cc1, np.arange(1, N + 1, dtype=np.float64)) - cc2

            # Calcul du coefficient de Hurst local
            mcord[tt - 1] = np.mean(np.abs(ddVd)**q) / np.mean(np.abs(VVVd)**q)

        # Régression linéaire sur le logarithme des échelles
        mx = np.mean(np.log10(x))
        SSxx = np.sum(np.log10(x)**2) - Tmax * mx**2
        my = np.mean(np.log10(mcord))
        SSxy = np.sum(np.multiply(np.log10(x), np.transpose(np.log10(mcord)))) - Tmax * mx * my

        # Stocke le résultat du coefficient de Hurst dans le tableau H
        H[k] = SSxy / SSxx
        k = k + 1

    # Calcule la moyenne des coefficients de Hurst sur toutes les fenêtres de temps et divise par q
    mH = np.mean(H) / q

    return mH  # Retourne le coefficient de Hurst moyen divisé par l'ordre q


def find_H(VA):
    n = VA.shape[0]

    delta_t=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    frequency = []
    for d in delta_t:
        frequency.append(n // d)
    
    #resampe with different frequencies
    asset_values={}
    for f in frequency:
        fasset_values = []
        days = []
        for i in range(n):
            if i % (n // f) == 0 :
                days.append(i)
        for day in days:
            fasset_values.append(VA[day])
        asset_values[f] = fasset_values

    #calculate Variance and Mean for every frequency
    Var = []
    for i, f in enumerate(frequency):
        Var.append(np.var(np.diff(np.log(asset_values[f]), n=1)) )

    Mean = []
    for i, f in enumerate(frequency):
        Mean.append(np.mean(np.diff(np.log(asset_values[f]), n=1)))
    var_tau = np.array(Var)


    # Transformation logarithmique
    log_delta_t = np.log(delta_t)
    log_var_tau = np.log(var_tau)

    fixed_intercept_log_sigma2 = np.log(var_tau[0]) # assuming delta = 1 otherwise H is here

    # Régression linéaire
    X = log_delta_t.reshape(-1, 1)
    y = log_var_tau - fixed_intercept_log_sigma2

    model = LinearRegression(fit_intercept=False)
    model.fit(X, y)
    #print('Regression score for H is ',model.score(X, y))

    # Coefficients de la régression
    slope = model.coef_[0]

    # Calcul de H
    H = slope / 2
    sigma_A = np.sqrt(var_tau[0]) * np.sqrt(int(n/delta_t[0]))
    #for i in range(len(delta_t)):
    #    print(np.sqrt(var_tau[i]) * np.sqrt(int(n/delta_t[i])))

    return sigma_A, H

Method 1 to obtain sigma and H. Sigma is obtained from H (more potential error).

In [44]:
def find_H_sigma(VA):
    n = VA.shape[0]

    delta_t=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    frequency = []
    for d in delta_t:
        frequency.append(n // d)
    
    #resampe with different frequencies
    asset_values={}
    for f in frequency:
        fasset_values = []
        days = []
        for i in range(n):
            if i % (n // f) == 0 :
                days.append(i)
        for day in days:
            fasset_values.append(VA[day])
        asset_values[f] = fasset_values

    #calculate Variance and Mean for every frequency
    Var = []
    for i, f in enumerate(frequency):
        Var.append(np.var(np.diff(np.log(asset_values[f]), n=1)) )

    Mean = []
    for i, f in enumerate(frequency):
        Mean.append(np.mean(np.diff(np.log(asset_values[f]), n=1)))
    var_tau = np.array(Var)


    # Transformation logarithmique
    log_delta_t = np.log(delta_t)
    log_var_tau = np.log(var_tau)

    fixed_intercept_log_sigma2 = np.log(var_tau[0]) # assuming delta = 1 otherwise H is here

    # Régression linéaire
    X = log_delta_t.reshape(-1, 1)
    y = log_var_tau - fixed_intercept_log_sigma2


    model = LinearRegression(fit_intercept=False)
    model.fit(X, y)
    #print('Regression score for H is ',model.score(X, y))

    # Coefficients de la régression
    slope = model.coef_[0]

    # Calcul de H
    H = slope / 2
    sigma_A = np.sqrt(var_tau[0]) * ((int(n/delta_t[0]))**(H))
    #for i in range(len(delta_t)):
    #    print(np.sqrt(var_tau[i]) * np.sqrt(int(n/delta_t[i])))

    return sigma_A, H

Method 2. The Calculation for Sigma appears different but it is in fact the same (from Cajueiro, Daniel O. and Fajardo, José, Volatility Estimation and Option Pricing with Fractional Brownian Motion (October 27, 2005). Available at SSRN: https://ssrn.com/abstract=837765 or http://dx.doi.org/10.2139/ssrn.837765)

In [45]:
def find_H_sigma_2(VA,T):

    ###Find H

    n = VA.shape[0]

    delta_t=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    frequency = []
    for d in delta_t:
        frequency.append(n // d)
    

    #resampe with different frequencies
    asset_values={}
    for f in frequency:
        fasset_values = []
        days = []
        for i in range(n):
            if i % (n // f) == 0 :
                days.append(i)
        for day in days:
            fasset_values.append(VA[day])
        asset_values[f] = fasset_values

    #calculate Variance and Mean for every frequency
    Var = []
    for i, f in enumerate(frequency):
        Var.append(np.var(np.diff(np.log(asset_values[f]), n=1)) )

    Mean = []
    for i, f in enumerate(frequency):
        Mean.append(np.mean(np.diff(np.log(asset_values[f]), n=1)))
    var_tau = np.array(Var)

    
    # Transformation logarithmique
    log_delta_t = np.log(delta_t)
    log_var_tau = np.log(var_tau)

    fixed_intercept_log_sigma2 = np.log(var_tau[0]) # assuming delta = 1 otherwise H is here

    # Régression linéaire
    X = log_delta_t.reshape(-1, 1)
    y = log_var_tau - fixed_intercept_log_sigma2

    model = LinearRegression(fit_intercept=False)
    model.fit(X, y)
    #print('Regression score for H is ',model.score(X, y))

    # Coefficients de la régression
    slope = model.coef_[0]

    # Calcul de H
    H = slope / 2


    #Calcul Sigma
    Z=np.log(VA)

    DZ=Z[1:]-Z[:-1]
    y=((n/T)**H)*DZ

    sigma_est= np.sqrt((n/(n-1))*np.var(y))

    return sigma_est,H

Comparison Methods

In [49]:
def sigma_error(true_sigma,mu,H,n,n_it=10, T=1):
    
    f = FBM(n, hurst=H, length=T, method='daviesharte')
    fbm_sample = f.fbm()
    times = f.times()
    VA=evol_va(mu,true_sigma,fbm_sample,times,H)
    sigma_1=find_H_sigma(VA)[0]
    sigma_2=find_H_sigma_2(VA,T)[0]

    err1=np.abs(sigma_1-true_sigma)
    err2=np.abs(sigma_2-true_sigma)
    return(sigma_1,sigma_2,err1,err2)

def H_error(sigma_a,mu,H,n,n_it=50):

    exp_h=np.zeros(n_it)
    err_h=np.zeros(n_it)

    
    for i in range (n_it):
        f = FBM(n, hurst=H, length=1, method='daviesharte')
        fbm_sample = f.fbm()
        times = f.times()
        exp_h[i]=find_H(evol_va(mu,sigma_a,fbm_sample,times,H))[1]
        err_h[i]=np.abs(exp_h[i]-H)

    return (np.mean(exp_h),np.mean(err_h))

def find_sigma(VA,H,T):

    n = VA.shape[0]
    X=np.log(VA)

    DX=X[1:]-X[:-1]
    y=((n/T)**H)*DX

    sigma_est= np.sqrt((n/(n-1))*np.var(y))

    return sigma_est

def sigma_error2(true_sigma,mu,H,n,sigma_a,n_it=10, T=1):

    f = FBM(n, hurst=H, length=T, method='daviesharte')
    fbm_sample = f.fbm()
    times = f.times()
    VA=evol_va(mu,sigma_a,fbm_sample,times,H)
    sigma_al=find_sigma(VA,H,T)

    err_sigma=np.abs(sigma_al-true_sigma)
    return(sigma_al,err_sigma)


Fonctions d'affichage

In [ ]:
def plot_distance_to_real_sigma(sigma_a=0.3, H=0.6, mu=0):
    sigma_al=[]
    err_sigma=[]
    liste_n=[]

    for n in range(100,1000,10):
        liste_n.append(n)
        s,e=sigma_error(sigma_a,mu,H,n,n_it=10)
        sigma_al.append(s)
        err_sigma.append(e)

    plt.plot(liste_n,sigma_al,label="sigma from algorithm")
    plt.plot(liste_n,err_sigma,label="distance to real sigma_A")
    plt.legend()

def plot_distance_to_real_H(sigma_a=0.3, H=0.6, mu=0):
    exp_h=[]
    err_h=[]
    liste_n=[]

    for n in range(100,1000,10):
        liste_n.append(n)
        s,e=H_error(sigma_a,mu,H,n,n_it=10)
        exp_h.append(s)
        err_h.append(e)

    plt.plot(liste_n,exp_h,label="H from algorithm")

    plt.plot(liste_n,err_h,label="absolute distance to real H")
    plt.legend()

def plot_distance_to_real_sigma_2(sigma_a=0.4, H=0.6, mu=0, T=1):
    sigma_al=[]
    err_sigma=[]
    liste_n=[]
    r_s=[]

    for n in range(10,252,10):
        liste_n.append(n)
        s,e=sigma_error2(sigma_a,mu,H,n,sigma_a, n_it=10)
        sigma_al.append(s)
        err_sigma.append(e)
        r_s.append(sigma_a)

    plt.plot(liste_n,sigma_al,label="sigma from algorithm")
    plt.plot(liste_n,r_s,label="Sigma_A reel")

    plt.plot(liste_n,err_sigma,label="distance to real sigma_A")
    plt.legend()

def plot_sigma_comparison(sigma_a=0.3, H=0.6, mu=0, T=1):
    
    sigma_1=[]
    sigma_2=[]
    err_sigma1=[]
    err_sigma2=[]

    liste_n=[]
    r_s=[]

    for n in range(50,5000,10):
        liste_n.append(n)
        s1,s2,e1,e2=sigma_error(sigma_a,mu,H,n,n_it=10)
        sigma_1.append(s1)
        sigma_2.append(s2)

        #err_sigma.append(e)
        r_s.append(sigma_a)

    plt.plot(liste_n,sigma_1,label="sigma from method 1")
    plt.plot(liste_n,r_s,label="Sigma_A reel")

    plt.plot(liste_n,sigma_2,label="sigma from method 2")
    plt.legend()

#### **Implémentation**

In [ ]:
sigma_a = 0.3
H = 0.6
mu = 0
T = 1
plot_distance_to_real_sigma(sigma_a, H, mu)
plot_distance_to_real_H(sigma_a, H, mu)
plot_distance_to_real_sigma_2(sigma_a, H, mu, T)
plot_sigma_comparison(sigma_a, H, mu, T)

### Mouvements brownien Gris

#### Méthodes

In [ ]:
def mb_gris(s,t,n_steps,H):

    times=np.linspace(s,t,n_steps)
    e = np.random.normal(size=n_steps)

    mbg=((times-s)**H)*e
    return(times,mbg)

def simulate_asset_price(S0, r, sigma, T, n_steps, n_it):
    dt = T / n_steps
    nudt = (r - 0.5 * sigma**2) * dt
    sigsdt = sigma * np.sqrt(dt)

    S = np.zeros((n_it, n_steps + 1))
    S[:, 0] = S0

    for i in range(n_it):
        #Draw of random walk
        e = np.random.normal(size=n_steps)
        for k in range(1, n_steps + 1):
            S[i, k] = S[i, k - 1] * np.exp(nudt + sigsdt * e[k-1])

    return S


#### **Implémentation**